In [250]:
file = open('fc_bounds.txt', 'r').read()

In [251]:
file_divided_in_fc = file.split('&')
file_divided_in_fc = file_divided_in_fc[1:]
print(file_divided_in_fc)

['ABST_FC_1\n[-0.5584585583211368, -1.5061802297201452, -1.4390591269781212, -1.4235034968571536, -0.6237085982879199]\n[0.5584585583211368, 1.5061802297201452, 1.4390591269781212, 1.4235034968571536, 0.6237085982879199]\n', 'ABST_FC_2\n[0.0, 1.0797846926900288e-16, 1.2017333152994098e-16, 9.385787957085258e-17, 1.7729012419367521e-16]\n[-4.93844574705746e-17, 0.0, 0.0, 0.0, 0.0]\n[1.9209077044631106e-16, 0.0, -2.9676673756646354e-17, 1.1428196351982678e-16, 1.8329239960589427e-16]\n[-6.66416065488723e-17, -1.2496174463519832e-16, 0.0, -1.8580961993233087e-16, -5.402102577470034e-18]\n[0.0, 2.970731281130589e-16, 1.0409417720321672e-16, 2.7429065615919796e-16, 3.6714344711409147e-16]\n[-5.981181763893281e-17, -7.012431658971704e-17, 1.0409417720321672e-16, 0.0, -5.406077638063864e-18]\n[-2.3382790794991945e-16, 1.285673944939913e-16, -5.197967365746996e-17, -1.0618033784886657e-16, -1.6043408798421535e-16]\n[0.0, 1.285673944939913e-16, 0.0, -6.198501819838282e-17, 0.0]\n[-1.67844737527

In [252]:
upper_bounds_per_layer = dict()
lower_bounds_per_layer = dict()

#print(file_divided_in_fc[0])
for block_of_rows in file_divided_in_fc:
    rows_of_bounds = block_of_rows.split('\n')
    rows_of_bounds = [x for x in rows_of_bounds if x != '']
    key = rows_of_bounds[0]
    rows_of_bounds = rows_of_bounds[1:]

    upper_bounds_per_layer[key] = list()
    lower_bounds_per_layer[key] = list()

    for i in range(len(rows_of_bounds)):
        rows_of_bounds[i] = rows_of_bounds[i].replace('[', '').replace(']', '')

        if (i%2) == 0:
            lower_bounds_per_layer[key].append([float(x) for x in rows_of_bounds[i].split(',')])
        else:
            upper_bounds_per_layer[key].append([float(x) for x in rows_of_bounds[i].split(',')])

    #upper_bounds_per_layer[rows_of_bounds[0]] =

    #bounds_per_layer[rows_of_bounds[0]] = list_of_list(','.join(rows_of_bounds[1:]))

In [253]:
print(upper_bounds_per_layer)


{'ABST_FC_1': [[0.5584585583211368, 1.5061802297201452, 1.4390591269781212, 1.4235034968571536, 0.6237085982879199]], 'ABST_FC_2': [[-4.93844574705746e-17, 0.0, 0.0, 0.0, 0.0], [-6.66416065488723e-17, -1.2496174463519832e-16, 0.0, -1.8580961993233087e-16, -5.402102577470034e-18], [-5.981181763893281e-17, -7.012431658971704e-17, 1.0409417720321672e-16, 0.0, -5.406077638063864e-18], [0.0, 1.285673944939913e-16, 0.0, -6.198501819838282e-17, 0.0], [0.8864409085942128, 2.4414051766306797e-16, 1.700910341448769e-16, 0.06671636296862522, 2.62251735430268e-16], [-5.299758751840462e-17, 5.302036680752635e-18, 8.289882205444211e-17, 0.0, -1.5443976269318394e-16], [0.025154246085682522, 8.174015595299564e-17, 0.042926066272242464, 0.09224158023637524, 0.015896874622265272], [-4.4089474873026377e-16, 1.9455540192393313e-16, 0.0, 1.670487392310157e-16, 3.4006934949588464e-16], [6.814230120528202e-18, 7.542635327046968e-17, 6.693125784693993e-19, 4.165794870135799e-18, 4.706225459440142e-18], [5.482

In [254]:
import pandas as pd
import csv
import numpy as np
from itertools import zip_longest
import math
from pynever.strategies.bound_propagation_gimelli.bounds_menager import MyBoundsManager
from pynever.strategies.bound_propagation_elena.verification.bounds.boundsmanagerelena import BoundsManagerElena


def print_multiple_csv(abst_network, prop, bound_propagation_type, path1=None, path2=None):
    if bound_propagation_type==0:
        pass
    if bound_propagation_type == 1:
        gimelli_bounds_menager = MyBoundsManager(abst_network, prop)
        gimelli_symb_bounds, gimelli_numeric_bounds = gimelli_bounds_menager.compute_bounds()
        print_to_csv(gimelli_numeric_bounds, path1)

    elif bound_propagation_type == 2:
        elena_bounds_menager = BoundsManagerElena(abst_network, prop)
        elena_symb_bounds, elena_numeric_bounds = elena_bounds_menager.compute_bounds()
        print_to_csv(elena_numeric_bounds, path2)

    elif bound_propagation_type == 3:
        gimelli_bounds_menager = MyBoundsManager(abst_network, prop)
        gimelli_symb_bounds, gimelli_numeric_bounds = gimelli_bounds_menager.compute_bounds()
        elena_bounds_menager = BoundsManagerElena(abst_network, prop)
        elena_symb_bounds, elena_numeric_bounds, post_bounds = elena_bounds_menager.compute_bounds()
        print_to_csv(gimelli_numeric_bounds, path1)
        print_to_csv(elena_numeric_bounds, path2)
        print_to_csv(post_bounds, path2.replace(".csv", "_symb.csv"))


def fill_with_nan(lst, length):
    result = lst.copy()
    while len(result) < length:
        result.append(math.nan)
    return result


def print_to_csv(data_dict: dict, file_path):
    to_print = list()
    header = list()
    length_list = list()
    for key in data_dict.keys():
        key_lower = key + "_lower"
        key_upper = key + "_upper"
        header.append(key_lower)
        header.append(key_upper)
        to_print.append(data_dict[key].lower.tolist())
        to_print.append(data_dict[key].upper.tolist())
        length_list.append(len(data_dict[key].lower.tolist()))

    max_length = max(length_list)
    for i in range(len(to_print)):
        to_print[i] = fill_with_nan(to_print[i], max_length)

    with open(file_path, mode='w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(header)
        for row in zip(*to_print):
            writer.writerow(row)





In [255]:
def compact(data_dict, upper=False):
    bounds_dict = dict()
    list_of_length = list()
    for key in data_dict.keys():
        list_of_lists = data_dict[key]
        list_of_length.append(max(list_of_lists, key=len))
        if upper:
            bounds_dict[key] = [max(x) for x in zip(*list_of_lists)]
        else:
            bounds_dict[key] = [min(x) for x in zip(*list_of_lists)]

    print(list_of_length)
    return bounds_dict, len(list_of_length)

In [256]:
bound_dict, max_len = compact(upper_bounds_per_layer)
print(bound_dict)
print(max_len)

[[0.5584585583211368, 1.5061802297201452, 1.4390591269781212, 1.4235034968571536, 0.6237085982879199], [-4.93844574705746e-17, 0.0, 0.0, 0.0, 0.0], [-1.6778095755666063e-15, 0.0]]
{'ABST_FC_1': [0.5584585583211368, 1.5061802297201452, 1.4390591269781212, 1.4235034968571536, 0.6237085982879199], 'ABST_FC_2': [-1.5697001833314657e-15, -3.7137733656510523e-16, -6.122067694021879e-17, -3.1490371658091263e-16, -1.5443976269318394e-16], 'ABST_FC_3': [-4.269463998365538e-15, -4.14131986290373e-15]}
[0.5584585583211368, 1.5061802297201452, 1.4390591269781212, 1.4235034968571536, 0.6237085982879199]
